# Dataset trainer 

In [ ]:
from models import *
from dataset import *
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm, tqdm_notebook
device = "cuda" if torch.cuda.is_available() else "cpu"

## Model loader
This part of code loads the model and contains optional state file loader.

In [ ]:
model = SpaghettiModel().to(device)
model.load_state_dict(torch.load('models/spaghettiAlPestoETonno.pth'))
model.eval()

dataset = SensorCameraDataset(pesto1 + tonno )
train_size = round(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
sample = train_dataset[0][0]


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [ ]:
# from torchsummary import summary
# device = "cuda"
# from models import *

# model = SpaghettiModel().to(device)

# summary(model, (3,64,64))


### Trainging
In this part the model is trained using the training dataset

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
history = torch.zeros((1,AnelliModel.HISTORY_LEN)).to(device)
for epoch in tqdm_notebook(range(15)):
    for sample, label in train_loader:
        sample = sample.to(device=device, dtype=torch.float)
        label = label.to(device=device, dtype=torch.float)

        optimizer.zero_grad()   # zero the gradient buffers
        output = model(sample)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()    # Does the update
        print(loss.item())


### Confusion matrix
Here, confusion matrix is generated from the test dataset

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plot

y_pred = [3]
y_true = [3]

for sample, labels in test_loader:
    sample = sample.to(device=device, dtype=torch.float)
    labels = labels.to(device=device, dtype=torch.float)

    output = model(sample)

    output = (torch.max(output, 1)[1]).data.cpu().numpy()
    y_pred.extend(output) # Save Prediction
    labels = (torch.max(labels,1)[1]).data.cpu().numpy()
    y_true.extend(labels) # Save Truth

classes = (0,1,2,3)

cf_matrix = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])
plot.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plot.show()


### Save model

In [ ]:
torch.save(model.state_dict(), "models/spaghettiAlPestoETonno.pth")